# Yield Curve

The visualization displays the daily U.S. Treasury yield curve from one month bills to 30 year bonds. Used web scraper to pull in the data.

In [1]:
# Import notebook libraries and dependencies
import os
import requests
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup

In [2]:
# Scrape U.S. Treasury data
soup = BeautifulSoup(requests.get('https://data.treasury.gov/feed.svc/DailyTreasuryYieldCurveRateData').text,'lxml')
table = soup.find_all('m:properties')
tbondvalues = []
for i in table:
    tbondvalues.append([i.find('d:new_date').text[:10],i.find('d:bc_1month').text,i.find('d:bc_2month').text,i.find('d:bc_3month').text,i.find('d:bc_6month').text,i.find('d:bc_1year').text,i.find('d:bc_2year').text,i.find('d:bc_3year').text,i.find('d:bc_5year').text,i.find('d:bc_10year').text,i.find('d:bc_20year').text,i.find('d:bc_30year').text])
ustcurve = pd.DataFrame(tbondvalues,columns=['date','1m','2m','3m','6m','1y','2y','3y','5y','10y','20y','30y'])

In [3]:
# Clean data
ustcurve.iloc[:,1:] = ustcurve.iloc[:,1:].apply(pd.to_numeric)#/100
ustcurve['date'] = pd.to_datetime(ustcurve['date'])
yield_curve=ustcurve.set_index('date').sort_index(ascending=False)
yield_curve.head()

,1m,2m,3m,6m,1y,2y,3y,5y,10y,20y,30y
date,,,,,,,,,,,
2020-08-25,0.08,0.1,0.11,0.12,0.13,0.15,0.18,0.3,0.69,1.18,1.39
2020-08-24,0.09,0.1,0.12,0.12,0.14,0.14,0.18,0.28,0.65,1.14,1.35
2020-08-21,0.07,0.09,0.1,0.12,0.13,0.16,0.16,0.27,0.64,1.13,1.35
2020-08-20,0.08,0.09,0.11,0.13,0.12,0.13,0.15,0.26,0.65,1.15,1.38
2020-08-19,0.07,0.09,0.11,0.12,0.13,0.14,0.17,0.29,0.68,1.2,1.42


In [4]:
# Set today's date and plot the yield curve

def get_yield_curve_vis():
    today = list(yield_curve.iloc[0])
    yield_curve_vis = px.line(yield_curve, x=['1m', '2m', '3m', '6m', '1y', '2y', '3y', '5y', '10y', '20y', '30y'], 
                              y=today,title='U.S. Treasury Yield Curve (%)')
    yield_curve_vis.update_layout(
        xaxis_title="Maturity",
        yaxis_title="Treasury Yield")
    
    return(yield_curve_vis)